In [14]:
import pandas as pd

In [46]:
df=pd.read_csv(r"C:\Users\Nidhi\Desktop\myntra\dataverse_files\fashion data on Instagram.csv")

In [47]:
df.head()

,UserId,Followings,Followers,MediaCount,BrandName,BrandCategory,Hashtags,Caption,ImgURL,Likes,...,NumberOfPeople,NumberOfFashionProduct,Anger,Contempt,Disgust,Fear,Happiness,Neutral,Sadness,Surprise
0,1171579752855683619_212070047,518.0,9840.0,541,abercrombie,High street,"beautiful, summer, fashion, love, cute, food, ...","We were born to be REAL, not to be perfect. ...",https://scontent.cdninstagram.com/t51.2885-15/...,97,...,0.931486,3.42211,0.083862,0.015089,0.000583,0.000026,0.000017,0.890586,0.009657,0.000181
1,1171594777274371222_176762322,7333.0,2300.0,272,abercrombie,High street,"teen, model, brunette, selfie, hollister, snap...","Gotta run, but first, let me take selfie. Me...",https://scontent.cdninstagram.com/t51.2885-15/...,94,...,0.466329,2.91971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,91,...,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,1171407552643586413_581125501,131.0,605.0,106,abercrombie,High street,"alexandani, cute, llbean, beanboots, anthropol...",dress to impress\U0001f457,https://scontent.cdninstagram.com/t51.2885-15/...,94,...,1.116500,4.69096,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,1171508187966229230_2797323089,132.0,456.0,56,abercrombie,High street,"abercrombieandfitch, shopping, love, hollister...",#love #shopping #shoppen #hollister #abercro...,https://scontent.cdninstagram.com/t51.2885-15/...,9,...,0.393063,1.45586,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [48]:
# converting unix dates to yyyy-mm-dd
df['Date'] = pd.to_datetime(df['CreationTime'], unit='s').dt.strftime('%Y-%m-%d')

In [49]:
df=df.dropna()

In [50]:
#convert encoded bytes to readable
def decode_unicode(text):
    return bytes(text, 'utf-8').decode('unicode_escape')

df['Hashtags'] = df['Hashtags'].str.split(', ').apply(lambda tags: [decode_unicode(tag) for tag in tags])

print(df['Hashtags'])

0        [beautiful, summer, fashion, love, cute, food,...
1        [teen, model, brunette, selfie, hollister, sna...
2        [alexandani, cute, llbean, beanboots, anthropo...
3        [alexandani, cute, llbean, beanboots, anthropo...
4        [abercrombieandfitch, shopping, love, holliste...
                               ...                        
24747    [เสื้อเท่ๆ, เสื้อใส่สบาย, เสื้อไปทะเล, เสื้อสว...
24748    [siambrandname, topshopthailand, somethingboud...
24749    [siambrandname, coach, peony, zara, รับตัวแทน,...
24750    [siambrandname, coach, peony, zara, รับตัวแทน,...
24751    [siambrandname, topshopthailand, somethingboud...
Name: Hashtags, Length: 24589, dtype: object


In [53]:
#get useful columns
df_required = df[['Date', 'Hashtags']]

In [55]:
# separate hashtags
df_exploded = df_required.explode('Hashtags')

In [56]:
# get date in datetime format
df_required['Date'] = pd.to_datetime(df_required['Date'])

C:\Users\Nidhi\AppData\Local\Temp\ipykernel_25240\1242471149.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_required['Date'] = pd.to_datetime(df_required['Date'])


In [63]:
# Convert the index to datetime
df_exploded.index = pd.to_datetime(df_exploded.index)

# groupby weekly trends
hashtag_trends = df_exploded.groupby('Hashtags').resample('W-Mon').size().reset_index(name='Count')

In [65]:
# set date as index and the single tags as columns
hashtag_trends_pivot = hashtag_trends.pivot(index='Date', columns='Hashtags', values='Count').fillna(0)

In [67]:
# precent difference, 1,,0,-1 returned
hashtag_trends_pivot_growth = hashtag_trends_pivot.pct_change().fillna(0)

In [75]:
# filter to get dates after 2015
filtered_hashtag_trends_pivot_growth = hashtag_trends_pivot_growth[hashtag_trends_pivot_growth.index >= "2015-01-01"]

In [78]:
# calculate mean of tags present over 7 day window
moving_average_window = 7
hashtag_moving_avg = hashtag_trends_pivot.rolling(window=moving_average_window).mean()

# find weekly difference
hashtag_moving_avg_diff = hashtag_moving_avg.diff()

In [79]:
# if difference is greater than zero then the hashtag trended over weeks (get average weeks lasted)
short_term_trending_tags = {}
for column in hashtag_moving_avg_diff.columns:

    if hashtag_moving_avg_diff[column].iloc[-1] > 0:
        short_term_trending_tags[column] = hashtag_moving_avg_diff[column].iloc[-1]

In [80]:
# sort by most weekly trending
sorted_trending_tags = sorted(short_term_trending_tags.items(), key=lambda x: x[1], reverse=True)

In [81]:
# Display the short-term trending hashtags
print("Short-term Trending Hashtags:")
for tag, trend_strength in sorted_trending_tags:
    print(f"{tag}: {trend_strength:.2f}")

Short-term Trending Hashtags:
fashion: 394.00
chanel: 246.57
ootd: 229.71
zara: 211.71
coach: 185.43
prada: 178.57
style: 171.71
siambrandname: 166.14
dior: 157.29
sbn: 149.43
hermes: 130.43
louisvuitton: 130.14
topshop: 123.57
gucci: 121.43
mango: 115.14
michaelkors: 108.29
forever21: 107.29
katespade: 107.00
love: 107.00
lynaround: 104.43
kloset: 102.00
lyn: 97.43
uniqlo: 94.00
urbanoutfitters: 93.14
stellamccartney: 92.29
fendi: 91.00
calvinklein: 90.71
outfit: 90.29
jcrew: 89.86
marcjacobs: 86.29
acnestudios: 82.14
shopping: 80.71
milin: 80.14
ตามหา: 79.29
fashionblogger: 79.00
sale: 78.43
americanapparel: 78.29
shoes: 76.57
instafashion: 74.86
nike: 74.71
lv: 74.57
givenchy: 73.71
gap: 72.29
alexandermcqueen: 71.86
instagood: 68.57
valentino: 64.00
streetstyle: 63.43
weloveshopping: 62.43
viviennewestwood: 61.71
hollister: 61.00
adidas: 60.86
disaya: 60.43
alexanderwang: 59.71
burberry: 59.14
followme: 56.14
jaspal: 55.29
clothes: 55.14
celine: 55.00
dubai: 54.57
paulsmith: 54.00
